In [1]:
#importing the required libraries and packages for the future use we can import them when ever necessary or we can import them
#when necessary

import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import WordPunctTokenizer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
from bs4 import BeautifulSoup
import os
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn import model_selection, preprocessing, metrics, svm, ensemble

In [2]:
#importing the train and test sets using pandas as dataframes

In [4]:
df_train=pd.read_csv("C:/Users/rogunda/Desktop/analytics_vidhya/train_E6oV3lV.csv")

In [8]:
df_test=pd.read_csv("C:/Users/rogunda/Desktop/analytics_vidhya/test_tweets_anuFYb8.csv")

In [9]:
#the first thing wew would want to see are the size of our datasets and check if there are any null values present in them 

In [10]:
df_train.shape #gives us number of rows and columns in that order 

(31962, 3)

In [11]:
df_test.shape 

(17197, 2)

In [12]:
df_test.head(5)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [13]:
df_train.describe()

,id,label
count,31962.000000,31962.000000
mean,15981.500000,0.070146
std,9226.778988,0.255397
min,1.000000,0.000000
25%,7991.250000,0.000000
50%,15981.500000,0.000000
75%,23971.750000,0.000000
max,31962.000000,1.000000


In [14]:
#though describe doesn't make any sense to text data .I ran it anyway so that we can see if there are any null values luckily 
#no null values

In [15]:
df_train.head(2)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...


In [16]:
#As we can see from one problem is that @user is like a prefix for most of the tweets , we should also remove other unecessary
#things such as special characters,alphanumeric,html tags etc

In [17]:
#let us write a function to achieve this goals
def clean_tweet(text):
    text = re.sub(r"http\S+", "", text) #It says remove characters when "http" is encountered unttil a space is encountered
                                        #thankfully links don't have spaces
    text = BeautifulSoup(text, 'lxml').get_text()
    #This will help us get text from any links that have been included some of the keywords in link smight contain useful info
    
    
    text=re.sub("\S*\d\S*", "", text).strip() # This means remove all the words which have numbers in them followed by 
                                              #or preceeded by any number of non space character
        
    
    text = re.sub('[^A-Za-z#]+', ' ', text) # Here I am replacing all the characters which are not alphabets or white space
    text=text.replace("user","")
    text = ' '.join(e.lower() for e in text.split() if len(e.lower())>2)
    text = re.sub(r"\s+"," ", text, flags = re.I).strip() # I got many spaces in between removing them and also strip() to 
                                                          #leading and trailing spaces
    return text
    

In [18]:
num=len(df_train)
clean_train_tweets=[]
for i in range(num):
    clean_train_tweets.append(clean_tweet(df_train['tweet'][i]))
    
#basically here for every tweet in the dataset we call the clean_tweet dataset

In [19]:
print(clean_train_tweets[:10])

['when father dysfunctional and selfish drags his kids into his dysfunction #run', 'thanks for #lyft credit can use cause they don offer wheelchair vans pdx #disapointed #getthanked', 'bihday your majesty', '#model love take with all the time', 'factsguide society now #motivation', 'huge fan fare and big talking before they leave chaos and pay disputes when they get there #allshowandnogo', 'camping tomorrow danny', 'the next school year the year for exams can think about that #school #exams #hate #imagine #actorslife #revolutionschool #girl', 'won love the land #allin #cavs #champions #cleveland #clevelandcavaliers', 'welcome here']


In [20]:
num=len(df_test)
clean_test_tweets=[]
for i in range(num):
    clean_test_tweets.append(clean_tweet(df_test['tweet'][i]))

In [21]:
print(clean_test_tweets[:10])

['#studiolife #aislife #requires #passion #dedication #willpower find #newmaterials', '#white #supremacists want everyone see the new #birds #movie and here why', 'safe ways heal your #acne #altwaystoheal #healthy #healing', 'the and the cursed child book for reservations already yes where when #harrypotter #pottermore #favorite', '#bihday amazing hilarious #nephew eli ahmir uncle dave loves you and misses', 'choose #momtips', 'something inside dies eyes ness #smokeyeyes #tired #lonely #sof #grunge', '#finished#tattoo#inked#ink#loveit #thanks#aleeee', 'will never understand why dad left when was young #deep #inthefeels', '#delicious #food #lovelife #capetown mannaepicure #resturant']


In [22]:
train_clean = pd.DataFrame(clean_train_tweets,columns=['tweet'])
train_clean['label'] = df_train.label
train_clean['id'] = df_train.id
test_clean = pd.DataFrame(clean_test_tweets,columns=['tweet'])
test_clean['id'] = df_test.id
#forming dataframes from lists

In [23]:
train_clean.head(5)

,tweet,label,id
0,when father dysfunctional and selfish drags hi...,0,1
1,thanks for #lyft credit can use cause they don...,0,2
2,bihday your majesty,0,3
3,#model love take with all the time,0,4
4,factsguide society now #motivation,0,5


In [24]:
tokenized_tweet = train_clean['tweet'].apply(lambda x: x.split())
#tokenizing is done so that we can perform stemming

In [25]:
tokenized_tweet.head()

0    [when, father, dysfunctional, and, selfish, dr...
1    [thanks, for, #lyft, credit, can, use, cause, ...
2                              [bihday, your, majesty]
3           [#model, love, take, with, all, the, time]
4              [factsguide, society, now, #motivation]
Name: tweet, dtype: object

In [26]:
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
#porterstemmer is used for stemming in this case

In [27]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    
train_clean['tweet'] = tokenized_tweet

In [28]:
train_clean.head(5)
#here we can see how words stemmed 

,tweet,label,id
0,when father dysfunct and selfish drag hi kid i...,0,1
1,thank for #lyft credit can use caus they don o...,0,2
2,bihday your majesti,0,3
3,#model love take with all the time,0,4
4,factsguid societi now #motiv,0,5


In [29]:
tokenized_tweet1 = test_clean['tweet'].apply(lambda x: x.split())

In [30]:
tokenized_tweet1 = tokenized_tweet1.apply(lambda x: [stemmer.stem(i) for i in x])
for i in range(len(tokenized_tweet1)):
    tokenized_tweet1[i] = ' '.join(tokenized_tweet1[i])
    
test_clean['tweet'] = tokenized_tweet1

In [31]:
test_clean.head(5)

,tweet,id
0,#studiolif #aislif #requir #passion #dedic #wi...,31963
1,#white #supremacist want everyon see the new #...,31964
2,safe way heal your #acn #altwaystoh #healthi #...,31965
3,the and the curs child book for reserv alreadi...,31966
4,#bihday amaz hilari #nephew eli ahmir uncl dav...,31967


In [32]:
train_clean.head(5)

,tweet,label,id
0,when father dysfunct and selfish drag hi kid i...,0,1
1,thank for #lyft credit can use caus they don o...,0,2
2,bihday your majesti,0,3
3,#model love take with all the time,0,4
4,factsguid societi now #motiv,0,5


In [40]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(train_clean['tweet'],train_clean['label'])


In [41]:
y_test.shape

(7991,)

In [36]:
train_clean.shape

(31962, 3)

In [45]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', stop_words='english', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(x_train)
xtest_tfidf =  tfidf_vect.transform(x_test)

In [43]:
def train_model(classifier,x_train,x_test,y_train,y_test):
    # fit the training dataset on the classifier
    classifier.fit(x_train,y_train)
    
    # predict the labels on validation dataset
    y_pred= classifier.predict(x_test)    

    return metrics.f1_score(y_test,y_pred)

In [46]:
accuracy = train_model(ensemble.ExtraTreesClassifier(n_estimators=300), xtrain_tfidf, xtest_tfidf,y_train,y_test)
print("Extremely Randomized Trees, WordLevel TF-IDF: ", accuracy)

Extremely Randomized Trees, WordLevel TF-IDF:  0.6833855799373041


In [54]:
#Now working on the testdata setoriginally provided in the problem statement
x_train=train_clean['tweet']
y_train=train_clean['label']
x_test=test_clean['tweet']
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', stop_words='english', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
tfidf_vect.fit(test_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(x_train)
xtest_tfidf =  tfidf_vect.transform(x_test)

In [55]:
classifier=ensemble.ExtraTreesClassifier(n_estimators=300)

In [56]:
classifier.fit(xtrain_tfidf,y_train)
y_pred=classifier.predict(xtest_tfidf)

In [57]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [58]:
y_pred.shape

(17197,)

In [63]:
pred_data={'id':df_test['id'],'label':y_pred}

In [64]:
df_pred_data=pd.DataFrame(data=pred_data)

In [65]:
df_pred_data.to_csv("twitter_predictions11.csv", index=False)